# Power Line Detection via Background Noise Removal

In [9]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import os

In [10]:
os.chdir(r"C:\Users\VArri\Documents\PowerLines\images\visuel\cropped")
print(os.getcwd())
path = os.getcwd()

C:\Users\VArri\Documents\PowerLines\images\visuel\cropped


In [78]:
lst = os.listdir()
imageslst = list()
labelslst = list()
labels = list()

for lis in lst:
    
    if (lis.endswith('.jpg') or lis.endswith('.jpeg')):
        
        filename, extension = lis.split('.')
        filetxt = '.'.join((filename, 'txt'))
        
        if os.path.exists(filetxt):
            imageslst.append(os.path.join(path,lis))
            labelslst.append(os.path.join(path, filetxt))
            f = open(filetxt, "r")
            labels.append(int(f.read()))
            f.close()

# Convert to tensors
imagepaths = tf.convert_to_tensor(imageslst, dtype=tf.string)
labelpaths = tf.convert_to_tensor(labelslst, dtype=tf.string)
labels = tf.convert_to_tensor(labels, dtype=tf.int8)

# https://stackoverflow.com/questions/37340129/tensorflow-training-on-my-own-image
# step 2: create a dataset returning slices of `filenames`
dataset = tf.data.Dataset.from_tensor_slices((imagepaths, labels))

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=1)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

dataset = dataset.map(_parse_function)
# dataset = dataset.batch(2)

# step 4: create iterator and final input tensor
#iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
#images, labels = iterator.get_next()

In [79]:
dataset

<MapDataset shapes: ((None, None, 1), ()), types: (tf.float32, tf.int8)>

## D) CNN classifier

In [15]:
model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1, padding="valid", input_shape=[28,28,1]),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(12, kernel_size=5, strides=1, padding="valid"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(192),
    keras.layers.Dense(2, activation="softmax")
])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 12)          1812      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 12)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 192)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 192)               37056     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])